# Vulnerability Curves

Script to create Damage Index vs Local Wind Speed best-fit curves with lognormal cdf fit, based on least square optimisation.

Allow plots to be shown inline in notebook.

In [1]:
% matplotlib inline

Import required modules

In [2]:
import numpy as np
from numpy import ma

import csv

from pylab import *

import matplotlib.pyplot as plt

from scipy.stats import lognorm
from scipy.stats import weibull_min
from scipy.optimize import leastsq
from scipy.optimize import minimize

#import matplotlib.gridspec as gridspec

Enter path of csv file containing [UNIQUE FIELD IDENTIFIER, BUILDING TYPE, LOCAL WIND SPEED, DAMAGE INDEX, HAZIMP DAMAGE INDEX*, HAZIMP WIND SPEED (0.2s gust)*] data:

*Enter columns of 0 if HAZIMP not used

In [3]:
file_path = 'N:/climate_change/CHARS/G_Users/u70993/Yasi_redo/damage_index_data.csv'

Import file and process data.

In [4]:
file = open(file_path)
data = csv.reader(file)

wind_pts = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
DI_pts = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
wind_haz = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
DI_haz = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]

for row in data:
    for i in range(10):
        building_type = 'TYPE %i' %(i+1)
        if row[1] == building_type and float(row[2]) > 10:
            wind_pts[i].append(float(row[2]))
            DI_pts[i].append(float(row[3]))
            if i in [0,1,2,3,4,6,7,8,9] and row[5] != '0':
                DI_haz[i].append(float(row[4]))
                wind_haz[i].append(float(row[5])/1.15) #0.2s to 3s gust conversion (1.132+1.164)/2 from Holomes & Ginger 2012 Table 4
            

            if i in [3,4]:
                wind_pts[10].append(float(row[2]))
                DI_pts[10].append(float(row[3]))
                DI_haz[10].append(float(row[4]))
                wind_haz[10].append(float(row[5])/1.15)
            elif i in [0,1,2,6]:
                wind_pts[11].append(float(row[2]))
                DI_pts[11].append(float(row[3]))
                DI_haz[11].append(float(row[4]))
                wind_haz[11].append(float(row[5]))
            elif i == 5:
                wind_pts[12].append(float(row[2]))
                DI_pts[12].append(float(row[3]))
                DI_haz[12].append(float(row[4]))
                wind_haz[12].append(float(row[5])/1.15)
            
            elif i in [8,9]:
                wind_pts[13].append(float(row[2]))
                DI_pts[13].append(float(row[3]))
                DI_haz[13].append(float(row[4]))
                wind_haz[13].append(float(row[5])/1.15)            
            elif i == 7:
                wind_pts[14].append(float(row[2]))
                DI_pts[14].append(float(row[3]))
                DI_haz[14].append(float(row[4]))
                wind_haz[14].append(float(row[5])/1.15)


wind_pts[15] = wind_pts[10] + wind_pts[13]
DI_pts[15] = DI_pts[10] + DI_pts[13]
wind_pts[16] = wind_pts[11] + wind_pts[14]
DI_pts[16] = DI_pts[11] + DI_pts[14]


file.close()

Define minimise and residuals functions which will be used to calculate best fit.

In [5]:
def residuals(params, build_type):
    x = wind_pts[build_type]
    y = DI_pts[build_type]
    output = y - lognorm.cdf(x, params[0], scale = params[1])
    return output

def minimise(params, build_type):
    output = leastsq(residuals, params, args = build_type, full_output = 1, ftol=1E-10, xtol=1E-10, maxfev = 100000)
    return output




def function_exp(params,x):
    alpha_inv = 1/params[0]
    beta_exp = np.exp(params[1])
    function_exp = np.exp(-(x/beta_exp)**(alpha_inv))
    output = 1 - function_exp
    return output

def residuals_exp(params, build_type):
    x = wind_pts[build_type]
    y = DI_pts[build_type]
    output = y - function_exp(params,x)
    return output

def minimise_exp(params, build_type):
    output = leastsq(residuals_exp, params, args = build_type, full_output = 1, ftol=1E-10, xtol=1E-10, maxfev = 100000)
    return output



def frange(x, y, jump):
    output = []
    val = x
    while val<=y:
        output.append(val)
        val = round(val+jump,1)
    return output

Carry out minimisation process and create plots for each building type.

Save plots and plotting info log file in:
"H:\\\My Documents\Vulnerability Curves"

In [6]:
infopath = 'H:\\My Documents\\Vulnerability_Curves\\Vulnerability_Plot_Info.csv'
info = open(infopath,'w')

info.write('Building_Type,Log_Shape,Log_Scale,Log_R_Squared,Log_RMSD,Exp_Shape,Exp_Scale,Exp_R_Squared,Exp_RMSD')

# MINIMISATION PROCESS - Iterate thorugh building types

result = []
result_exp = []
bands = [[],[],[],[],[],[],[],[],[],[]]


for b in range(10):
    
    bands[b]= [[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    
    for pt in range(0,len(wind_pts[b])):
        section = int(wind_pts[b][pt]/5-4)
        bands[b][section].append(DI_pts[b][pt])
    
    # Create initial estimates for parameters (Weibull)
    
    shape_guess_exp = np.std(wind_pts[b])
    scale_guess_exp = np.log(1./shape_guess_exp)
    params_guess_exp = [shape_guess_exp, scale_guess_exp]
    
    
    # Carry out minimisation process (Weibull)
    
    result_exp.append(minimise_exp(params_guess_exp,b))

    
    # Create initial estimates for parameters (lognorm)

    shape_guess = np.std(wind_pts[b])
    scale_guess = np.mean(np.log(shape_guess))    
    params_guess = [shape_guess, scale_guess]

    # Carry out minimisation process (lognorm)
    
    result.append(minimise(params_guess,b))
    
    # Calcualte Error
        
    error_exp = residuals_exp(result_exp[b][0],b)
    R_squared_exp = 1 - np.sum(np.square(error_exp)) / np.sum( np.square( DI_pts[b] - np.mean(DI_pts[b]) ) )
    RMSD_exp = np.sqrt(np.sum(np.square(error_exp))/(len(error_exp)-2))
    
    error = residuals(result[b][0],b)
    R_squared = 1 - np.sum(np.square(error)) / np.sum( np.square( DI_pts[b] - np.mean(DI_pts[b]) ) )
    RMSD = np.sqrt(np.sum(np.square(error))/(len(error)-2))
    
    # PLOT RESULTS
    
    fig = plt.figure(figsize=(8,12), dpi=1000)
    
    # Subplot 1 - Vulnerability Curve
    
    fig1 = subplot(2,1,1)
    
    if b == 0:
        hazimp_alpha = 0.1476
        hazimp_beta = 3.9768
    elif b in [1,2,6]:
        hazimp_alpha = 0.1457
        hazimp_beta = 3.9844
    elif b == 3:
        hazimp_alpha = 0.1129
        hazimp_beta = 4.3397
    elif b == 4:
        hazimp_alpha = 0.1057
        hazimp_beta = 4.2734
    
    hazimp_domain = frange(0,115,.1)
    hazimp_domain[:] = [x/1.15 for x in hazimp_domain] # conversion from 0.2s to 3s gust
    
    plot(
        frange(0,100,.1),lognorm.cdf(frange(0,100,.1),result[b][0][0], scale = result[b][0][1]),'g',
        frange(0,100,.1),function_exp(result_exp[b][0],frange(0,100,.1)),'k',
        markersize=10,lw=2)
    
    plot(
        frange(0,100,.1),lognorm.cdf(frange(0,100,.1),result[b][0][0], scale = result[b][0][1])+RMSD,'g:',
        frange(0,100,.1),function_exp(result_exp[b][0],frange(0,100,.1))+RMSD_exp,'k:',
        lw=1)
    
    
    
    if b in [0,1,2,3,4,6]: 
        
        plot(
        hazimp_domain,function_exp([hazimp_alpha, hazimp_beta],frange(0,115,.1)),'m',
        markersize=10,lw=2)       
        
        boxplot(bands[b], positions = (22.5,27.5,32.5,37.5,42.5,47.5,52.5,57.5,62.5,67.5,72.5,77.5,82.5,87.5),
        widths= 5, whis = 2)
        
        fig1.legend(["lognorm cdf", "weibull cdf", "1 RMSD (lognorm)", "1 RMSD (weibull)", "HAZIMP CURVE", "Observations"],loc=2)
    
    else:
        
        boxplot(bands[b], positions = (22.5,27.5,32.5,37.5,42.5,47.5,52.5,57.5,62.5,67.5,72.5,77.5,82.5,87.5),
        widths= 5, whis = 2)
        
        fig1.legend(["lognorm cdf", "weibull cdf","1 RMSD (lognorm)", "1 RMSD (weibull)", "Observations"],loc=2)
    
    
    plot(
        frange(0,100,.1),lognorm.cdf(frange(0,100,.1),result[b][0][0], scale = result[b][0][1])-RMSD,'g:',
        frange(0,100,.1),function_exp(result_exp[b][0],frange(0,100,.1))-RMSD_exp,'k:',
        lw=1)
    
    if result[b][0][1]>1e10:
        print 'Error - Building Type %i: lognorm.cdf least squares minimisation process failed to converge.' %(b+1)
    
    title = 'Vulnerability Curve (Least Squares Fit) - Buiding Type %i' %(b+1)
    fig1.set_title(title)
    fig1.set_xlabel('Wind Speed (3s Gust) [m/s]')
    fig1.set_ylabel('Damage Index')
    fig1.set_xlim((0,100))
    fig1.set_ylim((0,1))
    fig1.set_xticks([20, 40, 60, 80, 100])
    
    # subplot 2 - Residuals Plot
    
    fig2 =subplot(2,1,2)
    
    title = 'Residuals Plot - Buiding Type %i' %(b+1)
    fig2.set_title(title)
    fig2.set_xlabel('Wind Speed (3s Gust [m/s]')
    fig2.set_ylabel('Residual Damage Index')
    fig2.set_xlim((0,100))
    fig2.set_ylim((-1,1))
    
    plot(wind_pts[b],error,'.g', markersize = 3)
    plot(wind_pts[b],error_exp,'.k', markersize = 3)
    plot([0,100],[RMSD,RMSD],'g-.')
    plot([0,100],[RMSD_exp,RMSD_exp],'k:')
    plot([0,100],[-RMSD,-RMSD],'g-.')
    plot([0,100],[-RMSD_exp,-RMSD_exp],'k:')
    plot([0,100],[0,0],'k')
    
    
    fig2.legend(["lognorm","weibull", "1 RMSD (lognorm)","1 RMSD (weibull)"],loc=2)
    
    # Save Figure
    
    imgpath = 'H:\\My Documents\\Vulnerability_Curves\\Vulnerability_Plot_BT%i.png' %(b+1)
    fig.savefig(imgpath, dpi=300)
    
    # Enter data into info file
    
    info.write('\n%i, %f, %f, %f, %f, %f, %f, %f, %f'
               %(b+1, result[b][0][0], result[b][0][1], R_squared, RMSD, result_exp[b][0][0], result_exp[b][0][1], R_squared_exp, RMSD_exp))
    
info.close()

Plot lognorm.cdf curves for all building types on the same axes

In [7]:
fig = figure(figsize=(8,6), dpi=100)

fig1 = subplot(1,1,1)

fig1.set_xlabel('Wind Speed (3s Gust) [m/s]')
fig1.set_ylabel('Damage Index')
fig1.set_xlim((0,100))
fig1.set_ylim((0,1))

legend_str = []

for b in range(7):
    plot(frange(0,100,.1),function_exp(result_exp[b][0],frange(0,100,.1)))
    legend_str.append('Building Type %i'%(b+1))
    if result[b][0][1]>1e10:
        print 'Error - Building Type %i: lognorm.cdf least squares minimisation process failed to converge.' %(b+1)
    
fig1.set_title('Vulnerability Curves (Weibull) - All Buiding Types') 
fig1.legend(legend_str, loc = 2)

imgpath = 'H:\\My Documents\\Vulnerability_Curves\\Vulnerability_Curves_All_Types.png'
fig.savefig(imgpath, dpi=300)


Calculate and plot lognorm.cdf curves for typical code-compliant and uncompiant rediential buildings.

In [8]:
infopath = 'H:\\My Documents\\Vulnerability_Curves\\Vulnerability_Plot_Info.csv'
info = open(infopath,'a+')

result_1980=[]

error_1980 = [[],[],[]]
R_squared_1980 = [[],[],[]]
RMSD_1980 = [[],[],[]]


# DO CALCULATIONS AND PLOTTING

for c in [10,11,12]:
    
    # Create initial estimates for parameters
    
    shape_guess = np.std(wind_pts[c])
    loc_guess = np.log(1/shape_guess)
        
    params_guess = [shape_guess, loc_guess]
    
    # Carry out minimisation process
    
    result_1980.append(minimise_exp(params_guess,c))
    
    error_1980[c-10] = residuals_exp(result_1980[c-10][0],c)
    R_squared_1980[c-10] = 1 - np.sum(np.square(error_1980[c-10])) / np.sum( np.square( DI_pts[c] - np.mean(DI_pts[c]) ) )
    RMSD_1980[c-10] = np.sqrt(np.sum(np.square(error_1980[c-10]))/(len(error_1980[c-10])-2))
    
    info.write('\n%i, %f, %f, %f, %f, %f, %f, %f, %f'
               %(c+1,0 ,0 ,0 ,0 ,result_1980[c-10][0][0], result_1980[c-10][0][1], R_squared_1980[c-10],RMSD_1980[c-10]))

# PLOT
    
fig,ax = subplots(figsize=(8,6), dpi=600)
    
title = 'Vulnerability Curve (Weibull) - 1980 Building Code Compliance'
ax.set_title(title)
ax.set_xlabel('Wind Speed (3s Gust) [m/s]')
ax.set_ylabel('Damage Index')
ax.set_xlim((0,100))
ax.set_ylim((0,1))
    
    
ax.plot(frange(0,100,.1),function_exp(result_1980[0][0],frange(0,100,.1)),'-k',
        frange(0,100,.1),function_exp(result_1980[1][0],frange(0,100,.1)),'-r',
        frange(0,100,.1),function_exp(result_1980[2][0],frange(0,100,.1)),'-g',
        lw=2)

ax.plot(frange(0,100,.1),function_exp(result_1980[0][0],frange(0,100,.1))+RMSD_1980[0],'k--',
        frange(0,100,.1),function_exp(result_1980[1][0],frange(0,100,.1))+RMSD_1980[1],'r--',
        frange(0,100,.1),function_exp(result_1980[2][0],frange(0,100,.1))-RMSD_1980[2],'g--',
        frange(0,100,.1),function_exp(result_1980[0][0],frange(0,100,.1))-RMSD_1980[0],'k--',
        frange(0,100,.1),function_exp(result_1980[1][0],frange(0,100,.1))-RMSD_1980[1],'r--',
        frange(0,100,.1),function_exp(result_1980[2][0],frange(0,100,.1))+RMSD_1980[2],'g--',  
        lw=1)        


ax.plot(wind_pts[10],DI_pts[10], '.k',
        wind_pts[11],DI_pts[11],'.r',
        wind_pts[12],DI_pts[12], '.g',
        markersize = 1)

ax.legend(["Code Compliant Buildings", "Pre-Code Buildings", "Industrial Sheds", "1 RMSD (compliant)", "1 RMSD (pre-code)", "1 RMSD (sheds)"],loc=2)

info.close()

# Save Figure
    
imgpath = 'H:\\My Documents\\Vulnerability_Curves\\Vulnerability_Plot_Compliance.png'
fig.savefig(imgpath, dpi=300)
    

Functions for fragility curves

In [9]:
def residuals_fragility0(params, i_value, curve, total_sections_value, X_sections):
    
    x=[]
    x[:] = [(100/total_sections_value)/2 + m*(100/total_sections_value) for m in X_sections]
    
    y = []
    y[:] = [section_probs[i_value][m][curve] for m in range(len(x))]
    
    output = y - function_exp(params,x)
    return output

def minimise_fragility0(params, i_value, curve, total_sections_value,X_sections):
    output = leastsq(residuals_fragility0, params, args = (i_value, curve, total_sections_value,X_sections),
                     full_output = 1, ftol=1E-10, xtol=1E-10, maxfev = 100000)
    return output





def residuals_fragility( scale, shape, i_value, curve, total_sections_value, X_sections):
    
    x=[]
    x[:] = [(100/total_sections_value)/2 + m*(100/total_sections_value) for m in X_sections]
    
    y = []
    y[:] = [section_probs[i_value][m][curve] for m in range(len(x))]
    
    output = y - function_exp([shape,scale],x)
    return output

def minimise_fragility(scale, shape, i_value, curve, total_sections_value,X_sections):
    output = leastsq(residuals_fragility, scale, args = (shape, i_value, curve, total_sections_value,X_sections),
                     full_output = 1, ftol=1E-10, xtol=1E-10, maxfev = 100000)
    return output




Fragility Curves

In [10]:
section_bands = []
section_probs = []
section_x = []
result_fragility = []

infopath = 'H:\\My Documents\\Vulnerability_Curves\\Fragility_Plot_Info.csv'
info = open(infopath,'w')

info.write('Building_Type,\
\0.02_Shape,0.02_Scale,0.02_RMSD,\
\0.12_Shape,0.12_Scale,0.12_RMSD,\
\0.50_Shape,0.50_Scale,0.50_RMSD,\
\0.90_Shape,0.90_Scale,0.90_RMSD')


error_frag = [[],[],[],[]]
R_squared_frag = [[],[],[],[]]
RMSD_frag = [[],[],[],[]]

## post 1980

for i in [0]:
    
    total_sections = 20  # must be a factor of 100
    curve_shape_multiplier = [.04,.035] # use to alter shaopes of DI=0.12, DI=0.5 and DI=0.9 curves
    
    section_bands.append([])
    section_probs.append([])
    section_x.append([])
    result_fragility.append([[],[],[],[]])
    
    b=i+10
    
    wind = wind_pts[b]
    DI = DI_pts[b]
    
    section_bands[i][:] = [ [0,0,0,0,0] for element in range(total_sections) ]
    
    for pt in range(len(wind)):
        
        section = int(wind[pt]/(100/total_sections))        
        if section == total_sections:
            section -= 1
        
        section_bands[i][section][0] += 1
        
        if DI[pt] >= 0.02:
            section_bands[i][section][1] += 1
        if DI[pt] >= 0.12:
            section_bands[i][section][2] += 1
        if DI[pt] >= 0.50:
            section_bands[i][section][3] += 1
        if DI[pt] >= 0.90:
            section_bands[i][section][4] += 1
    
    for k in range(len(section_bands[i])): 
        if section_bands[i][k][0]!=0:
            section_probs[i].append([])
            section_x[i].append(k)
            section_probs[i][-1][:] = [ float(section_bands[i][k][j])/float(section_bands[i][k][0]) for j in range(1,5)]
    
    for k in range(100):
        section_x[i].append(total_sections*1.5) #1.2
        section_probs[i].append([1,1,1,1])
        section_x[i].append(total_sections*.3) #.3
        section_probs[i].append([0,0,0,0])

    shape_guess_exp = .2
    scale_guess_exp = 4
    params_guess_exp = [shape_guess_exp, scale_guess_exp]

    # Carry out minimisation process (Weibull)
    
    result_fragility[i][0] = minimise_fragility0( params_guess_exp , i, 0, total_sections, section_x[i])
    shape_true = result_fragility[i][0][0][0]
###    
    result_fragility[i][1:] = [ minimise_fragility( scale_guess_exp, shape_true-multiplier*curve_shape_multiplier[i], i, curve, total_sections, section_x[i] )
                              for (multiplier,curve) in [(1,1),(3.1,2),(3,3)] ]
    
    
    # Error
    error_frag[0] = residuals_fragility0(result_fragility[i][0][0],i,0, total_sections, section_x[i])
    RMSD_frag[0] = np.sqrt(np.sum(np.square(error_frag[0][:-200]))/(len(error_frag[0][:-200])-2))
    
###    
    for (multiplier,curve) in [(1,1),(3.1,2),(3,3)]:
        error_frag[curve] = residuals_fragility(result_fragility[i][curve][0],shape_true-multiplier*curve_shape_multiplier[i],i,curve, total_sections, section_x[i])
        RMSD_frag[curve] = np.sqrt(np.sum(np.square(error_frag[curve][:-200]))/(len(error_frag[curve][:-200])-1))
    
    
    info.write('\n%i, %f, %f, %f, %f, %f, %f, %f, %f, %f, %f, %f, %f'
               %(b+1,result_fragility[i][0][0][0] ,result_fragility[i][0][0][1] ,RMSD_frag[0],shape_true-1*curve_shape_multiplier[i],result_fragility[i][1][0],RMSD_frag[1],shape_true-2*curve_shape_multiplier[i],result_fragility[i][2][0],RMSD_frag[2],shape_true-3*curve_shape_multiplier[i],result_fragility[i][3][0],RMSD_frag[3]))        
    
    # PLOT RESULTS
    
    fig = plt.figure(figsize=(8,6), dpi=1000)
    
    # Subplot 1 - Vulnerability Curve
        
    fig1 = subplot(1,1,1)
    
    xx=[]
    xx[:] = [(100/total_sections)/2 + m*(100/total_sections) for m in section_x[i]]
    yy = []
  
    for curve in range(4):
        
        yy.append([])
        yy[curve][:] = [section_probs[i][n][curve] for n in range(len(xx))]
###
    multiplier = [1,3.1,3]
        
    plot(
        frange(0,100,.1),function_exp(result_fragility[i][0][0],frange(0,100,.1)), '-g',
        frange(0,100,.1),function_exp([shape_true-multiplier[0]*curve_shape_multiplier[i],result_fragility[i][1][0]],frange(0,100,.1)), '-b',
        frange(0,100,.1),function_exp([shape_true-multiplier[1]*curve_shape_multiplier[i],result_fragility[i][2][0]],frange(0,100,.1)), '-r',
        frange(0,100,.1),function_exp([shape_true-multiplier[2]*curve_shape_multiplier[i],result_fragility[i][3][0]],frange(0,100,.1)), '-k',
        xx[:-200],yy[3][:-200],'.k', [-1],[-1], '--k',
        xx[:-200],yy[0][:-200],'.g',
        xx[:-200],yy[1][:-200],'.b',
        xx[:-200],yy[2][:-200],'.r',   
        lw=2,markersize=15)
    
    plot(
        frange(0,100,.1),function_exp(result_fragility[i][0][0],frange(0,100,.1))+RMSD_frag[0], '--g',
        frange(0,100,.1),function_exp([shape_true-multiplier[0]*curve_shape_multiplier[i],result_fragility[i][1][0]],frange(0,100,.1))+RMSD_frag[1], '--b',
        frange(0,100,.1),function_exp([shape_true-multiplier[1]*curve_shape_multiplier[i],result_fragility[i][2][0]],frange(0,100,.1))+RMSD_frag[2], '--r',
        frange(0,100,.1),function_exp([shape_true-multiplier[2]*curve_shape_multiplier[i],result_fragility[i][3][0]],frange(0,100,.1))+RMSD_frag[3], '--k',
        frange(0,100,.1),function_exp(result_fragility[i][0][0],frange(0,100,.1))-RMSD_frag[0], '--g',
        frange(0,100,.1),function_exp([shape_true-multiplier[0]*curve_shape_multiplier[i],result_fragility[i][1][0]],frange(0,100,.1))-RMSD_frag[1], '--b',
        frange(0,100,.1),function_exp([shape_true-multiplier[1]*curve_shape_multiplier[i],result_fragility[i][2][0]],frange(0,100,.1))-RMSD_frag[2], '--r',
        frange(0,100,.1),function_exp([shape_true-multiplier[2]*curve_shape_multiplier[i],result_fragility[i][3][0]],frange(0,100,.1))-RMSD_frag[3], '--k',
        lw=1)
    
    
    
    fig1.set_xlabel('Wind Speed (3s Gust) [m/s]')
    fig1.set_ylabel('Probability of Damage')
    fig1.legend(['DI = 0.02', 'DI = 0.12','DI = 0.50','DI = 0.90', 'data point (5m/s section)', '1 RMSD'],loc=2)
    fig1.set_xlim((0,100))
    fig1.set_ylim((0,1))
    fig1.set_xticks([20, 40, 60, 80, 100])
    
    if i==0:
        title = 'Fragility Plot (Weibull) - Post 1980 Buildings (Code Compliant)'
        imgpath = 'H:\\My Documents\\Vulnerability_Curves\\Fragility_Plot_post1980.png'
    elif i==1:
        title = 'Fragility Plot (Weibull) - Pre 1980 Buildings'
        imgpath = 'H:\\My Documents\\Vulnerability_Curves\\Fragility_Plot_pre1980.png'
    
    fig1.set_title(title)
    fig.savefig(imgpath, dpi=600)

    
    
    
    
    
    
    
## pre 1980 




for i in [1]:
    
    total_sections = 20  # must be a factor of 100
    curve_shape_multiplier = [.01,.036] # use to alter shaopes of DI=0.12, DI=0.5 and DI=0.9 curves
    
    section_bands.append([])
    section_probs.append([])
    section_x.append([])
    result_fragility.append([[],[],[],[]])
    
    b=i+10
    
    wind = wind_pts[b]
    DI = DI_pts[b]
    
    section_bands[i][:] = [ [0,0,0,0,0] for element in range(total_sections) ]
    
    for pt in range(len(wind)):
        
        section = int(wind[pt]/(100/total_sections))        
        if section == total_sections:
            section -= 1
        
        section_bands[i][section][0] += 1
        
        if DI[pt] >= 0.02:
            section_bands[i][section][1] += 1
        if DI[pt] >= 0.12:
            section_bands[i][section][2] += 1
        if DI[pt] >= 0.50:
            section_bands[i][section][3] += 1
        if DI[pt] >= 0.90:
            section_bands[i][section][4] += 1
    
    for k in range(len(section_bands[i])): 
        if section_bands[i][k][0]!=0:
            section_probs[i].append([])
            section_x[i].append(k)
            section_probs[i][-1][:] = [ float(section_bands[i][k][j])/float(section_bands[i][k][0]) for j in range(1,5)]
    
    for k in range(100):
        section_x[i].append(total_sections*1.5) #1.2
        section_probs[i].append([1,1,1,1])
        section_x[i].append(total_sections*.3) #.3
        section_probs[i].append([0,0,0,0])

    shape_guess_exp = .2
    scale_guess_exp = 4
    params_guess_exp = [shape_guess_exp, scale_guess_exp]

    # Carry out minimisation process (Weibull)
    
    result_fragility[i][0] = minimise_fragility0( params_guess_exp , i, 0, total_sections, section_x[i])
    shape_true = result_fragility[i][0][0][0]
###    
    result_fragility[i][1:] = [ minimise_fragility( scale_guess_exp, shape_true-multiplier*curve_shape_multiplier[i], i, curve, total_sections, section_x[i] )
                              for (multiplier,curve) in [(1,1),(3,2),(3.2,3)] ]
    
    
    # Error
    error_frag[0] = residuals_fragility0(result_fragility[i][0][0],i,0, total_sections, section_x[i])
    RMSD_frag[0] = np.sqrt(np.sum(np.square(error_frag[0][:-200]))/(len(error_frag[0][:-200])-2))
    
###    
    for (multiplier,curve) in [(1,1),(3,2),(3.2,3)]:
        error_frag[curve] = residuals_fragility(result_fragility[i][curve][0],shape_true-multiplier*curve_shape_multiplier[i],i,curve, total_sections, section_x[i])
        RMSD_frag[curve] = np.sqrt(np.sum(np.square(error_frag[curve][:-200]))/(len(error_frag[curve][:-200])-1))
    
    
    info.write('\n%i, %f, %f, %f, %f, %f, %f, %f, %f, %f, %f, %f, %f'
               %(b+1,result_fragility[i][0][0][0] ,result_fragility[i][0][0][1] ,RMSD_frag[0],shape_true-1*curve_shape_multiplier[i],result_fragility[i][1][0],RMSD_frag[1],shape_true-2*curve_shape_multiplier[i],result_fragility[i][2][0],RMSD_frag[2],shape_true-3*curve_shape_multiplier[i],result_fragility[i][3][0],RMSD_frag[3]))        
    
    # PLOT RESULTS
    
    fig = plt.figure(figsize=(8,6), dpi=1000)
    
    # Subplot 1 - Vulnerability Curve
        
    fig1 = subplot(1,1,1)
    
    xx=[]
    xx[:] = [(100/total_sections)/2 + m*(100/total_sections) for m in section_x[i]]
    yy = []

    for curve in range(4):
        
        yy.append([])
        yy[curve][:] = [section_probs[i][n][curve] for n in range(len(xx))]
###
    multiplier = [1,3,3.2]
        
    plot(
        frange(0,100,.1),function_exp(result_fragility[i][0][0],frange(0,100,.1)), '-g',
        frange(0,100,.1),function_exp([shape_true-multiplier[0]*curve_shape_multiplier[i],result_fragility[i][1][0]],frange(0,100,.1)), '-b',
        frange(0,100,.1),function_exp([shape_true-multiplier[1]*curve_shape_multiplier[i],result_fragility[i][2][0]],frange(0,100,.1)), '-r',
        frange(0,100,.1),function_exp([shape_true-multiplier[2]*curve_shape_multiplier[i],result_fragility[i][3][0]],frange(0,100,.1)), '-k',
        xx[:-200],yy[3][:-200],'.k', [-1],[-1], '--k',
        xx[:-200],yy[0][:-200],'.g',
        xx[:-200],yy[1][:-200],'.b',
        xx[:-200],yy[2][:-200],'.r',   
        lw=2,markersize=15)
    
    plot(
        frange(0,100,.1),function_exp(result_fragility[i][0][0],frange(0,100,.1))+RMSD_frag[0], '--g',
        frange(0,100,.1),function_exp([shape_true-multiplier[0]*curve_shape_multiplier[i],result_fragility[i][1][0]],frange(0,100,.1))+RMSD_frag[1], '--b',
        frange(0,100,.1),function_exp([shape_true-multiplier[1]*curve_shape_multiplier[i],result_fragility[i][2][0]],frange(0,100,.1))+RMSD_frag[2], '--r',
        frange(0,100,.1),function_exp([shape_true-multiplier[2]*curve_shape_multiplier[i],result_fragility[i][3][0]],frange(0,100,.1))+RMSD_frag[3], '--k',
        frange(0,100,.1),function_exp(result_fragility[i][0][0],frange(0,100,.1))-RMSD_frag[0], '--g',
        frange(0,100,.1),function_exp([shape_true-multiplier[0]*curve_shape_multiplier[i],result_fragility[i][1][0]],frange(0,100,.1))-RMSD_frag[1], '--b',
        frange(0,100,.1),function_exp([shape_true-multiplier[1]*curve_shape_multiplier[i],result_fragility[i][2][0]],frange(0,100,.1))-RMSD_frag[2], '--r',
        frange(0,100,.1),function_exp([shape_true-multiplier[2]*curve_shape_multiplier[i],result_fragility[i][3][0]],frange(0,100,.1))-RMSD_frag[3], '--k',
        lw=1)
    
    
    
    fig1.set_xlabel('Wind Speed (3s Gust) [m/s]')
    fig1.set_ylabel('Probability of Damage')
    fig1.legend(['DI = 0.02', 'DI = 0.12','DI = 0.50','DI = 0.90', 'data point (5m/s section)', '1 RMSD'],loc=2)
    fig1.set_xlim((0,100))
    fig1.set_ylim((0,1))
    fig1.set_xticks([20, 40, 60, 80, 100])
    
    if i==0:
        title = 'Fragility Plot (Weibull) - Post 1980 Buildings (Code Compliant)'
        imgpath = 'H:\\My Documents\\Vulnerability_Curves\\Fragility_Plot_post1980.png'
    elif i==1:
        title = 'Fragility Plot (Weibull) - Pre 1980 Buildings'
        imgpath = 'H:\\My Documents\\Vulnerability_Curves\\Fragility_Plot_pre1980.png'
    
    fig1.set_title(title)
    fig.savefig(imgpath, dpi=600)

    
    
    
    
    
info.close()

Combine TC Larry and TC Yasi data

In [11]:
infopath = 'H:\\My Documents\\Vulnerability_Curves\\Vulnerability_Plot_Info.csv'
info = open(infopath,'a+')

result_LY = []
bands_LY = [[],[],[],[]]

error_LY = [[],[]]
R_squared_LY = [[],[]]
RMSD_LY = [[],[]]



for i, c in [(0,10),(1,11),(2,13),(3,14)]:
    bands_LY[i]= [[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    for pt in range(0,len(wind_pts[c])):
        section = int(wind_pts[c][pt]/5-4)
        bands_LY[i][section].append(DI_pts[c][pt])

        
for i,c in [(0,15),(1,16)]:
    
    # Create initial estimates for parameters
    
    shape_guess = np.std(wind_pts[c])
    loc_guess = np.log(1/shape_guess)
        
    params_guess = [shape_guess, loc_guess]

    # Carry out minimisation process
    
    result_LY.append(minimise_exp(params_guess,c))
    
    # Error
    
    error_LY[i] = residuals_exp(result_LY[i][0],c)
    R_squared_LY[i] = 1 - np.sum(np.square(error_LY[i])) / np.sum( np.square( DI_pts[c] - np.mean(DI_pts[c]) ) )
    RMSD_LY[i] = np.sqrt(np.sum(np.square(error_LY[i]))/(len(error_LY[i])-2))
    
    info.write('\n%i, %f, %f, %f, %f, %f, %f, %f, %f'
               %(c+1,0 ,0 ,0 ,0 ,result_LY[i][0][0], result_LY[i][0][1], R_squared_LY[i],RMSD_LY[i]))

    
    # PLOT RESULTS
    
    fig,ax = subplots(figsize=(8,6), dpi=600)
    
    
    ax.plot([],[],'-k',lw=2)
    ax.plot([],[],'-g',lw=2)
    ax.plot([],[],'-b',lw=2)
    ax.plot([],[],'-r',lw=2)
    
    
    if c == 15:
        bp1 = boxplot(bands_LY[0], positions = (22.5,27.5,32.5,37.5,42.5,47.5,52.5,57.5,62.5,67.5,72.5,77.5,82.5,87.5),
            widths= 5, whis = 2, patch_artist = True)
        
        bp2 = boxplot(bands_LY[2], positions = (22.5,27.5,32.5,37.5,42.5,47.5,52.5,57.5,62.5,67.5,72.5,77.5,82.5,87.5),
            widths= 5, whis = 2)
      
    if c == 16:
        bp1 =boxplot(bands_LY[1], positions = (22.5,27.5,32.5,37.5,42.5,47.5,52.5,57.5,62.5,67.5,72.5,77.5,82.5,87.5),
            widths= 5, whis = 2, patch_artist = True)
        
        bp2 = boxplot(bands_LY[3], positions = (22.5,27.5,32.5,37.5,42.5,47.5,52.5,57.5,62.5,67.5,72.5,77.5,82.5,87.5),
            widths= 5, whis = 2)
        
         
    
    setp(bp1['boxes'], color='blue')
    setp(bp1['whiskers'], color='blue') 
    setp(bp1['fliers'], color='blue')
    setp(bp1['medians'], color='white',lw=2)
    setp(bp1['caps'], color='blue')
    
    setp(bp2['boxes'], color='red')
    setp(bp2['whiskers'], color='red')
    setp(bp2['fliers'], color='red')
    setp(bp2['medians'], color='red',lw=2)
    setp(bp2['caps'], color='red')
    
    plot(
        frange(0,100,.1),function_exp(result_LY[i][0],frange(0,100,.1)),'k',
        markersize=10,lw=2)
    
    plot(
        frange(0,100,.1),function_exp(result_1980[i][0],frange(0,100,.1)),'g',
        markersize=10,lw=2)
    
    ax.plot(frange(0,100,.1),function_exp(result_LY[i][0],frange(0,100,.1))+RMSD_LY[i],'k--',
        frange(0,100,.1),function_exp(result_LY[i][0],frange(0,100,.1))-RMSD_LY[i],'k--',
        frange(0,100,.1),function_exp(result_1980[i][0],frange(0,100,.1))+RMSD_1980[i],'g--',
        frange(0,100,.1),function_exp(result_1980[i][0],frange(0,100,.1))-RMSD_1980[i],'g--',
        lw=1)  
    
    
    
    ax.set_xlabel('Wind Speed (3s Gust) [m/s]')
    ax.set_ylabel('Damage Index')
    ax.set_xlim((0,100))
    ax.set_ylim((0,1))
    ax.set_xticks([20, 40, 60, 80, 100])
    ax.legend(["Weibull - Combined", "Weibull - TC Yasi", "Boxplot - TC Yasi", "Boxplot - TC Larry"],loc=2)
    
    
    # Save Figure

    if c == 15:
        title = 'Vulnerability Curve - TC Yasi and TC Larry - Code Compliant Post 1980'
        imgpath = 'H:\\My Documents\\Vulnerability_Curves\\Vulnerability_Plot_YasiLarry_post1980.png'
    elif c ==16:
        title = 'Vulnerability Curve - TC Yasi and TC Larry - Pre 1980'
        imgpath = 'H:\\My Documents\\Vulnerability_Curves\\Vulnerability_Plot_YasiLarry_pre1980.png'

    ax.set_title(title)
    
    fig.savefig(imgpath, dpi=600)

info.close()

Fragility plots - TC Yasi and TC Larry Combined

In [12]:
section_bands = []
section_probs = []
section_x = []
result_fragility = []

infopath = 'H:\\My Documents\\Vulnerability_Curves\\Fragility_Plot_Info.csv'
info = open(infopath,'a+')


error_frag = [[],[],[],[]]
R_squared_frag = [[],[],[],[]]
RMSD_frag = [[],[],[],[]]


### post-1980

for i in [0]:
    
    total_sections = 20  # must be a factor of 100
    curve_shape_multiplier = [.033,.037] # use to alter shaopes of DI=0.12, DI=0.5 and DI=0.9 curves
    
    section_bands.append([])
    section_probs.append([])
    section_x.append([])
    result_fragility.append([[],[],[],[]])
    
    b=i+15
    
    wind = wind_pts[b]
    DI = DI_pts[b]
    
    section_bands[i][:] = [ [0,0,0,0,0] for element in range(total_sections) ]
    
    for pt in range(len(wind)):
        
        section = int(wind[pt]/(100/total_sections))        
        if section == total_sections:
            section -= 1
        
        section_bands[i][section][0] += 1
        
        if DI[pt] >= 0.02:
            section_bands[i][section][1] += 1
        if DI[pt] >= 0.12:
            section_bands[i][section][2] += 1
        if DI[pt] >= 0.50:
            section_bands[i][section][3] += 1
        if DI[pt] >= 0.90:
            section_bands[i][section][4] += 1
    
    for k in range(len(section_bands[i])): 
        if section_bands[i][k][0]!=0:
            section_probs[i].append([])
            section_x[i].append(k)
            section_probs[i][-1][:] = [ float(section_bands[i][k][j])/float(section_bands[i][k][0]) for j in range(1,5)]
    
    for k in range(100):
        section_x[i].append(total_sections*1.5) #1.2
        section_probs[i].append([1,1,1,1])
        section_x[i].append(total_sections*.3) #.3
        section_probs[i].append([0,0,0,0])

    shape_guess_exp = .2
    scale_guess_exp = 4
    params_guess_exp = [shape_guess_exp, scale_guess_exp]

    # Carry out minimisation process (Weibull)
    
    result_fragility[i][0] = minimise_fragility0( params_guess_exp , i, 0, total_sections, section_x[i])
    shape_true = result_fragility[i][0][0][0]
###    
    result_fragility[i][1:] = [ minimise_fragility( scale_guess_exp, shape_true-multiplier*curve_shape_multiplier[i], i, curve, total_sections, section_x[i] )
                              for (multiplier,curve) in [(1,1),(3.1,2),(3,3)] ]
    
    
    # Error
    error_frag[0] = residuals_fragility0(result_fragility[i][0][0],i,0, total_sections, section_x[i])
    RMSD_frag[0] = np.sqrt(np.sum(np.square(error_frag[0][:-200]))/(len(error_frag[0][:-200])-2))
    
###    
    for (multiplier,curve) in [(1,1),(3.1,2),(3,3)]:
        error_frag[curve] = residuals_fragility(result_fragility[i][curve][0],shape_true-multiplier*curve_shape_multiplier[i],i,curve, total_sections, section_x[i])
        RMSD_frag[curve] = np.sqrt(np.sum(np.square(error_frag[curve][:-200]))/(len(error_frag[curve][:-200])-1))
    
    info.write('\n%i, %f, %f, %f, %f, %f, %f, %f, %f, %f, %f, %f, %f'
               %(b+1,result_fragility[i][0][0][0] ,result_fragility[i][0][0][1] ,RMSD_frag[0],shape_true-1*curve_shape_multiplier[i],result_fragility[i][1][0],RMSD_frag[1],shape_true-2*curve_shape_multiplier[i],result_fragility[i][2][0],RMSD_frag[2],shape_true-3*curve_shape_multiplier[i],result_fragility[i][3][0],RMSD_frag[3]))        
    
    
    # PLOT RESULTS
    
    fig = plt.figure(figsize=(8,6), dpi=1000)

###
    multiplier =[1,3.1,3]
    
    # Subplot 1 - Vulnerability Curve
        
    fig1 = subplot(1,1,1)
    
    xx=[]
    xx[:] = [(100/total_sections)/2 + m*(100/total_sections) for m in section_x[i]]
    yy = []
    
    for curve in range(4):
        
        yy.append([])
        yy[curve][:] = [section_probs[i][n][curve] for n in range(len(xx))]
        
    plot(
        frange(0,100,.1),function_exp(result_fragility[i][0][0],frange(0,100,.1)), '-g',
        frange(0,100,.1),function_exp([shape_true-multiplier[0]*curve_shape_multiplier[i],result_fragility[i][1][0]],frange(0,100,.1)), '-b',
        frange(0,100,.1),function_exp([shape_true-multiplier[1]*curve_shape_multiplier[i],result_fragility[i][2][0]],frange(0,100,.1)), '-r',
        frange(0,100,.1),function_exp([shape_true-multiplier[2]*curve_shape_multiplier[i],result_fragility[i][3][0]],frange(0,100,.1)), '-k',
        xx[:-200],yy[3][:-200],'.k', [-1],[-1], '--k',
        xx[:-200],yy[0][:-200],'.g',
        xx[:-200],yy[1][:-200],'.b',
        xx[:-200],yy[2][:-200],'.r',   
        lw=2,markersize=15)
    
    plot(
        frange(0,100,.1),function_exp(result_fragility[i][0][0],frange(0,100,.1))+RMSD_frag[0], '--g',
        frange(0,100,.1),function_exp([shape_true-multiplier[0]*curve_shape_multiplier[i],result_fragility[i][1][0]],frange(0,100,.1))+RMSD_frag[1], '--b',
        frange(0,100,.1),function_exp([shape_true-multiplier[1]*curve_shape_multiplier[i],result_fragility[i][2][0]],frange(0,100,.1))+RMSD_frag[2], '--r',
        frange(0,100,.1),function_exp([shape_true-multiplier[2]*curve_shape_multiplier[i],result_fragility[i][3][0]],frange(0,100,.1))+RMSD_frag[3], '--k',
        frange(0,100,.1),function_exp(result_fragility[i][0][0],frange(0,100,.1))-RMSD_frag[0], '--g',
        frange(0,100,.1),function_exp([shape_true-multiplier[0]*curve_shape_multiplier[i],result_fragility[i][1][0]],frange(0,100,.1))-RMSD_frag[1], '--b',
        frange(0,100,.1),function_exp([shape_true-multiplier[1]*curve_shape_multiplier[i],result_fragility[i][2][0]],frange(0,100,.1))-RMSD_frag[2], '--r',
        frange(0,100,.1),function_exp([shape_true-multiplier[2]*curve_shape_multiplier[i],result_fragility[i][3][0]],frange(0,100,.1))-RMSD_frag[3], '--k',
        lw=1)
    
    
    
    fig1.set_xlabel('Wind Speed (3s Gust) [m/s]')
    fig1.set_ylabel('Probability of Damage')
    fig1.legend(['DI = 0.02', 'DI = 0.12','DI = 0.50','DI = 0.90', 'data point (5m/s section)', '1 RMSD'],loc=2)
    fig1.set_xlim((0,100))
    fig1.set_ylim((0,1))
    fig1.set_xticks([20, 40, 60, 80, 100])
    
    if i==0:
        title = 'Fragility Plot (Weibull) - Post 1980 (TC Yasi & TC Larry Combined)'
        imgpath = 'H:\\My Documents\\Vulnerability_Curves\\Fragility_Plot_post1980_combined.png'
    elif i==1:
        title = 'Fragility Plot (Weibull) - Pre 1980 Buildings (TC Yasi & TC Larry Combined)'
        imgpath = 'H:\\My Documents\\Vulnerability_Curves\\Fragility_Plot_pre1980_combined.png'
    
    fig1.set_title(title)
    fig.savefig(imgpath, dpi=600)

    
    
    
    
    
    
### pre-1980    
    
    
for i in [1]:
    
    total_sections = 20  # must be a factor of 100
    curve_shape_multiplier = [.01,.036] # use to alter shaopes of DI=0.12, DI=0.5 and DI=0.9 curves
    
    section_bands.append([])
    section_probs.append([])
    section_x.append([])
    result_fragility.append([[],[],[],[]])
    
    b=i+15
    
    wind = wind_pts[b]
    DI = DI_pts[b]
    
    section_bands[i][:] = [ [0,0,0,0,0] for element in range(total_sections) ]
    
    for pt in range(len(wind)):
        
        section = int(wind[pt]/(100/total_sections))        
        if section == total_sections:
            section -= 1
        
        section_bands[i][section][0] += 1
        
        if DI[pt] >= 0.02:
            section_bands[i][section][1] += 1
        if DI[pt] >= 0.12:
            section_bands[i][section][2] += 1
        if DI[pt] >= 0.50:
            section_bands[i][section][3] += 1
        if DI[pt] >= 0.90:
            section_bands[i][section][4] += 1
    
    for k in range(len(section_bands[i])): 
        if section_bands[i][k][0]!=0:
            section_probs[i].append([])
            section_x[i].append(k)
            section_probs[i][-1][:] = [ float(section_bands[i][k][j])/float(section_bands[i][k][0]) for j in range(1,5)]
    
    for k in range(100):
        section_x[i].append(total_sections*1.5) #1.2
        section_probs[i].append([1,1,1,1])
        section_x[i].append(total_sections*.3) #.3
        section_probs[i].append([0,0,0,0])

    shape_guess_exp = .2
    scale_guess_exp = 4
    params_guess_exp = [shape_guess_exp, scale_guess_exp]

    # Carry out minimisation process (Weibull)
    
    result_fragility[i][0] = minimise_fragility0( params_guess_exp , i, 0, total_sections, section_x[i])
    shape_true = result_fragility[i][0][0][0]
###    
    result_fragility[i][1:] = [ minimise_fragility( scale_guess_exp, shape_true-multiplier*curve_shape_multiplier[i], i, curve, total_sections, section_x[i] )
                              for (multiplier,curve) in [(1,1),(3,2),(3.2,3)] ]
    
    
    # Error
    error_frag[0] = residuals_fragility0(result_fragility[i][0][0],i,0, total_sections, section_x[i])
    RMSD_frag[0] = np.sqrt(np.sum(np.square(error_frag[0][:-200]))/(len(error_frag[0][:-200])-2))
    
###    
    for (multiplier,curve) in [(1,1),(3,2),(3.2,3)]:
        error_frag[curve] = residuals_fragility(result_fragility[i][curve][0],shape_true-multiplier*curve_shape_multiplier[i],i,curve, total_sections, section_x[i])
        RMSD_frag[curve] = np.sqrt(np.sum(np.square(error_frag[curve][:-200]))/(len(error_frag[curve][:-200])-1))
    
    info.write('\n%i, %f, %f, %f, %f, %f, %f, %f, %f, %f, %f, %f, %f'
               %(b+1,result_fragility[i][0][0][0] ,result_fragility[i][0][0][1] ,RMSD_frag[0],shape_true-1*curve_shape_multiplier[i],result_fragility[i][1][0],RMSD_frag[1],shape_true-2*curve_shape_multiplier[i],result_fragility[i][2][0],RMSD_frag[2],shape_true-3*curve_shape_multiplier[i],result_fragility[i][3][0],RMSD_frag[3]))        
    
    
    # PLOT RESULTS
    
    fig = plt.figure(figsize=(8,6), dpi=1000)

###
    multiplier =[1,3,3.2]
    
    # Subplot 1 - Vulnerability Curve
        
    fig1 = subplot(1,1,1)
    
    xx=[]
    xx[:] = [(100/total_sections)/2 + m*(100/total_sections) for m in section_x[i]]
    yy = []
    
    for curve in range(4):
        
        yy.append([])
        yy[curve][:] = [section_probs[i][n][curve] for n in range(len(xx))]
        
    plot(
        frange(0,100,.1),function_exp(result_fragility[i][0][0],frange(0,100,.1)), '-g',
        frange(0,100,.1),function_exp([shape_true-multiplier[0]*curve_shape_multiplier[i],result_fragility[i][1][0]],frange(0,100,.1)), '-b',
        frange(0,100,.1),function_exp([shape_true-multiplier[1]*curve_shape_multiplier[i],result_fragility[i][2][0]],frange(0,100,.1)), '-r',
        frange(0,100,.1),function_exp([shape_true-multiplier[2]*curve_shape_multiplier[i],result_fragility[i][3][0]],frange(0,100,.1)), '-k',
        xx[:-200],yy[3][:-200],'.k', [-1],[-1], '--k',
        xx[:-200],yy[0][:-200],'.g',
        xx[:-200],yy[1][:-200],'.b',
        xx[:-200],yy[2][:-200],'.r',   
        lw=2,markersize=15)
    
    plot(
        frange(0,100,.1),function_exp(result_fragility[i][0][0],frange(0,100,.1))+RMSD_frag[0], '--g',
        frange(0,100,.1),function_exp([shape_true-multiplier[0]*curve_shape_multiplier[i],result_fragility[i][1][0]],frange(0,100,.1))+RMSD_frag[1], '--b',
        frange(0,100,.1),function_exp([shape_true-multiplier[1]*curve_shape_multiplier[i],result_fragility[i][2][0]],frange(0,100,.1))+RMSD_frag[2], '--r',
        frange(0,100,.1),function_exp([shape_true-multiplier[2]*curve_shape_multiplier[i],result_fragility[i][3][0]],frange(0,100,.1))+RMSD_frag[3], '--k',
        frange(0,100,.1),function_exp(result_fragility[i][0][0],frange(0,100,.1))-RMSD_frag[0], '--g',
        frange(0,100,.1),function_exp([shape_true-multiplier[0]*curve_shape_multiplier[i],result_fragility[i][1][0]],frange(0,100,.1))-RMSD_frag[1], '--b',
        frange(0,100,.1),function_exp([shape_true-multiplier[1]*curve_shape_multiplier[i],result_fragility[i][2][0]],frange(0,100,.1))-RMSD_frag[2], '--r',
        frange(0,100,.1),function_exp([shape_true-multiplier[2]*curve_shape_multiplier[i],result_fragility[i][3][0]],frange(0,100,.1))-RMSD_frag[3], '--k',
        lw=1)
    
    
    
    fig1.set_xlabel('Wind Speed (3s Gust) [m/s]')
    fig1.set_ylabel('Probability of Damage')
    fig1.legend(['DI = 0.02', 'DI = 0.12','DI = 0.50','DI = 0.90', 'data point (5m/s section)', '1 RMSD'],loc=2)
    fig1.set_xlim((0,100))
    fig1.set_ylim((0,1))
    fig1.set_xticks([20, 40, 60, 80, 100])
    
    if i==0:
        title = 'Fragility Plot (Weibull) - Post 1980 (TC Yasi & TC Larry Combined)'
        imgpath = 'H:\\My Documents\\Vulnerability_Curves\\Fragility_Plot_post1980_combined.png'
    elif i==1:
        title = 'Fragility Plot (Weibull) - Pre 1980 Buildings (TC Yasi & TC Larry Combined)'
        imgpath = 'H:\\My Documents\\Vulnerability_Curves\\Fragility_Plot_pre1980_combined.png'
    
    fig1.set_title(title)
    fig.savefig(imgpath, dpi=600)    
    
    
    
info.close()

Fragility Curves - TC Yasi only (original unmodified shape)

In [13]:
section_bands = []
section_probs = []
section_x = []
result_fragility = []

infopath = 'H:\\My Documents\\Vulnerability_Curves\\Fragility_Plot_Info.csv'
info = open(infopath,'a+')


error_frag = [[],[],[],[]]
R_squared_frag = [[],[],[],[]]
RMSD_frag = [[],[],[],[]]




for i in range(2):
    
    total_sections = 20  # must be a factor of 100
    curve_shape_multiplier = [.0,.0] # use to alter shaopes of DI=0.12, DI=0.5 and DI=0.9 curves
    
    section_bands.append([])
    section_probs.append([])
    section_x.append([])
    result_fragility.append([[],[],[],[]])
    
    b=i+10
    
    wind = wind_pts[b]
    DI = DI_pts[b]
    
    section_bands[i][:] = [ [0,0,0,0,0] for element in range(total_sections) ]
    
    for pt in range(len(wind)):
        
        section = int(wind[pt]/(100/total_sections))        
        if section == total_sections:
            section -= 1
        
        section_bands[i][section][0] += 1
        
        if DI[pt] >= 0.02:
            section_bands[i][section][1] += 1
        if DI[pt] >= 0.12:
            section_bands[i][section][2] += 1
        if DI[pt] >= 0.50:
            section_bands[i][section][3] += 1
        if DI[pt] >= 0.90:
            section_bands[i][section][4] += 1
    
    for k in range(len(section_bands[i])): 
        if section_bands[i][k][0]!=0:
            section_probs[i].append([])
            section_x[i].append(k)
            section_probs[i][-1][:] = [ float(section_bands[i][k][j])/float(section_bands[i][k][0]) for j in range(1,5)]
    
#    if i ==0:
    for k in range(100):
        section_x[i].append(total_sections*0) #1.2
        section_probs[i].append([0,0,0,0])
        section_x[i].append(total_sections*0) #.3
        section_probs[i].append([0,0,0,0])
        
    shape_guess_exp = .2
    scale_guess_exp = 4
    params_guess_exp = [shape_guess_exp, scale_guess_exp]

    # Carry out minimisation process (Weibull)
    
    result_fragility[i][0] = minimise_fragility0( params_guess_exp , i, 0, total_sections, section_x[i])
    shape_true = result_fragility[i][0][0][0]
    
    result_fragility[i][1:] = [ minimise_fragility( scale_guess_exp, shape_true-curve*curve_shape_multiplier[i], i, curve, total_sections, section_x[i] )
                              for (multiplier,curve) in [(1,1),(2,2),(3,3)] ]
    
    
    # Error
    error_frag[0] = residuals_fragility0(result_fragility[i][0][0],i,0, total_sections, section_x[i])
    RMSD_frag[0] = np.sqrt(np.sum(np.square(error_frag[0][:-200]))/(len(error_frag[0][:-200])-2))
    
    
    for u in range(1,4):
        error_frag[u] = residuals_fragility(result_fragility[i][u][0],shape_true-u*curve_shape_multiplier[i],i,u, total_sections, section_x[i])
        RMSD_frag[u] = np.sqrt(np.sum(np.square(error_frag[u][:-200]))/(len(error_frag[u][:-200])-1))
    
    info.write('\n%i (original unmodified shape), %f, %f, %f, %f, %f, %f, %f, %f, %f, %f, %f, %f'
               %(b+1,result_fragility[i][0][0][0] ,result_fragility[i][0][0][1] ,RMSD_frag[0],shape_true-1*curve_shape_multiplier[i],result_fragility[i][1][0],RMSD_frag[1],shape_true-2*curve_shape_multiplier[i],result_fragility[i][2][0],RMSD_frag[2],shape_true-3*curve_shape_multiplier[i],result_fragility[i][3][0],RMSD_frag[3]))        
    
    
    # PLOT RESULTS
    
    fig = plt.figure(figsize=(8,6), dpi=1000)
    
    # Subplot 1 - Vulnerability Curve
        
    fig1 = subplot(1,1,1)
    
    xx=[]
    xx[:] = [(100/total_sections)/2 + m*(100/total_sections) for m in section_x[i]]
    yy = []
    
    for curve in range(4):
        
        yy.append([])
        yy[curve][:] = [section_probs[i][n][curve] for n in range(len(xx))]
        
    plot(
        frange(0,100,.1),function_exp(result_fragility[i][0][0],frange(0,100,.1)), '-g',
        frange(0,100,.1),function_exp([shape_true-1*curve_shape_multiplier[i],result_fragility[i][1][0]],frange(0,100,.1)), '-b',
        frange(0,100,.1),function_exp([shape_true-2*curve_shape_multiplier[i],result_fragility[i][2][0]],frange(0,100,.1)), '-r',
        frange(0,100,.1),function_exp([shape_true-3*curve_shape_multiplier[i],result_fragility[i][3][0]],frange(0,100,.1)), '-k',
        xx[:-200],yy[3][:-200],'.k', [-1],[-1], '--k',
        xx[:-200],yy[0][:-200],'.g',
        xx[:-200],yy[1][:-200],'.b',
        xx[:-200],yy[2][:-200],'.r',   
        lw=2,markersize=15)
    
    plot(
        frange(0,100,.1),function_exp(result_fragility[i][0][0],frange(0,100,.1))+RMSD_frag[0], '--g',
        frange(0,100,.1),function_exp([shape_true-1*curve_shape_multiplier[i],result_fragility[i][1][0]],frange(0,100,.1))+RMSD_frag[1], '--b',
        frange(0,100,.1),function_exp([shape_true-2*curve_shape_multiplier[i],result_fragility[i][2][0]],frange(0,100,.1))+RMSD_frag[2], '--r',
        frange(0,100,.1),function_exp([shape_true-3*curve_shape_multiplier[i],result_fragility[i][3][0]],frange(0,100,.1))+RMSD_frag[3], '--k',
        frange(0,100,.1),function_exp(result_fragility[i][0][0],frange(0,100,.1))-RMSD_frag[0], '--g',
        frange(0,100,.1),function_exp([shape_true-1*curve_shape_multiplier[i],result_fragility[i][1][0]],frange(0,100,.1))-RMSD_frag[1], '--b',
        frange(0,100,.1),function_exp([shape_true-2*curve_shape_multiplier[i],result_fragility[i][2][0]],frange(0,100,.1))-RMSD_frag[2], '--r',
        frange(0,100,.1),function_exp([shape_true-3*curve_shape_multiplier[i],result_fragility[i][3][0]],frange(0,100,.1))-RMSD_frag[3], '--k',
        lw=1)
    
    
    
    fig1.set_xlabel('Wind Speed (3s Gust) [m/s]')
    fig1.set_ylabel('Probability of Damage')
    fig1.legend(['DI = 0.02', 'DI = 0.12','DI = 0.50','DI = 0.90', 'data point (5m/s section)', '1 RMSD'],loc=2)
    fig1.set_xlim((0,100))
    fig1.set_ylim((0,1))
    fig1.set_xticks([20, 40, 60, 80, 100])
    
    if i==0:
        title = 'Fragility Plot (Weibull) - Post 1980 (TC Yasi only, Original Unmodified Shape)'
        imgpath = 'H:\\My Documents\\Vulnerability_Curves\\Fragility_Plot_post1980_unmodified.png'
    elif i==1:
        title = 'Fragility Plot (Weibull) - Pre 1980 Buildings (TC Yasi only, Original Unmodified Shape)'
        imgpath = 'H:\\My Documents\\Vulnerability_Curves\\Fragility_Plot_pre1980_unmodified.png'
    
    fig1.set_title(title)
    fig.savefig(imgpath, dpi=600)

info.close()